# Identify gaps in CARE service relative to unsheltered population
* Map number of CARE/CARE+ requests by tract (2017-2019)
* Map number of homeless counts by tract (2017-2019)
* Map difference: (total CARE/CARE+ for homeless encampments) - (total homelessness)

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import ipyleaflet
from ipyleaflet import Map, GeoData, LayersControl, basemaps
import json
import requests
from ipywidgets import link, FloatSlider, Text, HTML
from branca.colormap import linear
import boto3
import folium

In [2]:
catalog = intake.open_catalog('../catalogs/*.yml')

bucket_name = 's3://city-of-los-angeles-data-lake/public-health-dashboard/'

s3 = boto3.client('s3')

## Merge dfs

In [ ]:
pivot1 = m2.dissolve(by = ['id', 'clipped_area', 'full_area', 
                           'CD', 'pop', 'SPA', 'SD'], aggfunc = 'sum').reset_index().drop(columns = ['year'])

pivot1['UnsheltDensity'] = pivot1.totUnshelt / pivot1.clipped_area
pivot1['CARE_Unshelt'] = pivot1.apply(lambda row: row.homeless / row.totUnshelt if row.totUnshelt > 0 else np.nan, axis = 1)
pivot1['Unshelt_CARE'] = pivot1.apply(lambda row: row.totUnshelt / row.homeless if row.homeless > 0 else np.nan, axis = 1)

pivot1.to_file(driver = 'GeoJSON', filename = '../gis/homelessness_tract.geojson')
s3.upload_file('../gis/homelessness_tract.geojson', 'city-of-los-angeles-data-lake', 
               'public-health-dashboard/gis/intermediate/homelessness_tract.geojson')
"""

In [ ]:
pivot1 = gpd.read_file(f'{bucket_name}gis/intermediate/homelessness_tract.geojson')

In [ ]:
pivot1.head()

## Map homelessness

In [ ]:
tracts = pivot1.copy()
tracts['GEOID'] = tracts['id']
tracts = tracts.set_index('id')
tracts.head()

In [ ]:
# to_json converts the gdf to json. json.loads converts it into dictionary 
# geo_data can have many more columns than just geometry. Need to include other columns if you want to have the HTML popup include info.
geo_data = json.loads(tracts.to_json())

# Take what we want to map and turn it into a dictionary
# Can only include the key-value pair, the value you want to map, nothing more.
unsheltered = dict(zip(pivot1['id'].tolist(), pivot1['totUnshelt'].tolist()))
unshelt_density = dict(zip(pivot1['id'].tolist(), pivot1['UnsheltDensity'].tolist()))
unshelt_CARE = dict(zip(pivot1['id'].tolist(), pivot1['Unshelt_CARE'].tolist()))
CARE_unshelt = dict(zip(pivot1['id'].tolist(), pivot1['CARE_Unshelt'].tolist()))

In [ ]:
m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 10,
                  basemap = basemaps.CartoDB.Positron)

layer = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = unsheltered,
    colormap = linear.viridis,
    border_color = 'white',
    style = {'fillOpacity': 0.6, 'weight': 0.8, 'opacity': 0.6},
    hover_style = {'fillOpacity': 0.8},
    value_min = 0,
    value_max = 100
)

html = HTML(''' 
    Hover over a tract
''')

html.layout.margin = '0 px 10px 10px 10px'

def update_html(feature, id, **kwargs): 
    html.value = '''
        Census Tract:  
        <b>{}</b> <br>
        Total Unsheltered:
        {} 
    '''.format(id, feature['properties']['totUnshelt'])
 
    
layer.on_hover(update_html)

control = ipyleaflet.WidgetControl(widget = html, position = 'topright')
m.add_layer(layer)
m.add_control(control)

m

In [ ]:
m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 10,
                  basemap = basemaps.CartoDB.Positron)

layer = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = unshelt_density,
    colormap = linear.Oranges_08,
    border_color = 'white',
    style = {'fillOpacity': 0.7, 'weight': 0.8, 'opacity': 0.6},
    hover_style = {'fillOpacity': 0.8},
    value_min = 0,
    value_max = 500
)

html = HTML(''' 
    Hover over a tract
''')

html.layout.margin = '0 px 10px 10px 10px'

def update_html(feature, id, **kwargs): 
    html.value = '''
        Census Tract:  
        <b>{}</b> <br>
        Unsheltered per Square Mile:
        {} 
    '''.format(id, feature['properties']['UnsheltDensity'])
 
    
layer.on_hover(update_html)

control = ipyleaflet.WidgetControl(widget = html, position = 'topright')
m.add_layer(layer)
m.add_control(control)

m

In [ ]:
unshelt_df = pivot1[['id', 'totUnshelt']]
unshelt_df.rename(columns = {'id': 'GEOID'}, inplace = True)
#geo = pivot1[['id', 'geometry']]
 
m = folium.Map(
    location=[34.0536, -118.2427], 
    zoom_start=12, 
    tiles='cartodbpositron'
)

folium.Choropleth(
    geo_data=pivot1,
    name = 'choropleth',
    data=unshelt_df,
    columns=['GEOID', 'totUnshelt'],
    key_on='feature.properties.id',
    fill_color='BuPu',
    fill_opacity=0.95,
    line_opacity=0.8, line_color='white',
    bins=[0, 50, 100, 500, 4000],
    legend_name='Housing Units per Acre'
).add_to(m)


#m.add_child(folium.LayerControl())

m.save('map.html')